<a href="https://colab.research.google.com/github/DeepSmeag/ML_Colab/blob/main/AND%2BXORperceptrons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/DeepSmeag/ML_Colab

Cloning into 'ML_Colab'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 66 (delta 30), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (66/66), done.


First we load the project so we get access to the files. Then we read the files of interest

In [2]:
def read_file(file_path):
  dataset=[]
  with open(file_path, 'r') as f:
    lines = f.readlines()
    for line in lines:
      line = line.strip()
      parts = line.split(',')
      x0,x1,y1,y2 = float(parts[0]), float(parts[1]), float(parts[2]), float(parts[3])
      dataset.append((x0,x1,y1,y2))
    return dataset

dataset = read_file('ML_Colab/datasetAND+XOR.csv')
"""it could also work with numpy and/or pandas to read the whole csv file at the
same time, probably more efficient too
"""


'it could also work with numpy and/or pandas to read the whole csv file at the\nsame time, probably more efficient too\n'

We now have our dataset ready for use, next let's go and make the vectors for use with our network

In [21]:
import numpy as np
import csv
import pandas

In [4]:
#first layer is input, meaning we take the first 2 columns of the dataset
num_trains = len(dataset)
X = np.ones((num_trains,1)) 
#now we add the bias to every training instance
X = np.append(X, np.array(dataset)[:,0:2], axis=1)
# now we take the outputs
Y = np.array(dataset)[:,2:4]

In [82]:
#now we create Numpy arrays so that we have our weights
#the first layer
W1 = np.random.uniform(-1,1,(3,3))
#the second
W2 = np.random.uniform(-1,1,(4,2))
print("W1:\n",W1,"\n")
print("W2:\n",W2,"\n")

W1:
 [[-0.95586837  0.93349817  0.95214154]
 [ 0.06574888 -0.49372003 -0.66550179]
 [ 0.90333154 -0.65779367  0.34961274]] 

W2:
 [[ 0.6113833   0.85550519]
 [ 0.74877283  0.57283235]
 [-0.43315179  0.03552481]
 [-0.28665714  0.37042094]] 



In [ ]:
#Now we're getting to the fun stuff
#First, defining the necessary functions

def sigmoid(x):
  return 1/(1 + np.exp(-x))

#Forward Propagation
def forward_prop(X, W1, W2):
  #calculating the steps
  z2 = np.matmul(X, W1)
  a2 = sigmoid(z2)
  a2 = np.insert(a2, 3, np.ones((num_trains,)), axis = 1)
  z3 = np.matmul(a2, W2)
  y_hat = sigmoid(z3)
  return z2, a2, z3, y_hat



In [84]:
#Ok, so that was forward propagation
#Now, the NN should also be able to learn, so we have to implement Backprop
#Since we're guessing at best at the moment as to the correct outputs
#We have to define several functions for Backprop to work
#These include the partial derivatives with which we'll update the weights matrices W1 and W2
#And also the backprop itself which will go over epochs or whatever we may call it

# matrices sizes
# X = num_trains x 3
# W1 = 3 x 3
# Z2 = num_trains x 3
# A2 = num_trains x 4 by adding a collumn of ones at the end
# W2 = 4 x 2
# Z3 = num_trains x 2
# A3 = Y_hat = num_trains x 2


def sigmoid_gradient(x):
  return sigmoid(x)*(1-sigmoid(x))
def derivative(z2, a2, z3, y_hat, training_point):
  dz3 = sigmoid_gradient(z3(training_point, :)) # 1 x 2
  dw2 = np.matmul(a2(training_point, :).transpose(), dz3) # 4 x 2
  dz2 = sigmoid_gradient(z2(training_point, :)) # 1 x 3
  temp1 = np.matmul(dz3, W2.transpose()) # 1 x 4
  temp1 = np.delete(temp1, 3, 1) # drop last column because it's for bias, 1 x 3
  temp2 = np.multiply(temp1, dz2) # num_trains x 3
  dw1 = np.matmul(temp2.transpose(), X(training_point, :)) # 3 x 3
  return dw1, dw2

def backprop(num_trains, learning_rate, X, W1, W2):
  z2, a2, z3, y_hat = forward_prop(X, W1, W2)
  for training_point in range(num_trains):
    print("current_training_point: ", i+1, "\n")
    
    dw1, dw2 = derivative(z2, a2, z3, y_hat, training_point)
    W1 = np.subtract(W1, learning_rate * dw1)
    W2 = np.subtract(W2, learning_rate * dw2)
  with open("ML_Colab/AND+XORmodelW1.csv", 'w') as fw1:
    np.savetxt(fw1 , W1 , fmt='%s', delimiter=',')
  with open("ML_Colab/AND+XORmodelW2.csv", 'w') as fw2:
    np.savetxt(fw2 , W2 , fmt='%s', delimiter=',')

backprop(10, 0.1, X, W1, W2)

epoch:  1 

epoch:  2 

epoch:  3 

epoch:  4 

epoch:  5 

epoch:  6 

epoch:  7 

epoch:  8 

epoch:  9 

epoch:  10 



In [85]:
new_w1 = np.genfromtxt('ML_Colab/AND+XORmodelW1.csv', delimiter=',')

new_w2 = np.genfromtxt('ML_Colab/AND+XORmodelW2.csv', delimiter=',')

Null, Null, Null, y_hat = forward_prop(X, new_w1, new_w2)
y_hat = np.rint(y_hat)
print(y_hat)

[[0. 0.]
 [0. 0.]
 [0. 0.]
 ...
 [0. 0.]
 [0. 0.]
 [0. 0.]]


In [9]:
Y

array([[0., 0.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [1., 0.]])